In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
import numpy as np
import gc

2024-12-16 18:09:29.157924: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-16 18:09:29.168106: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734359969.179801   47599 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734359969.183383   47599 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-16 18:09:29.195762: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
df = pd.read_csv('passwords.csv')

In [3]:
df = df.dropna(subset=['password'])

In [4]:
label_encoder = LabelEncoder()
df['strength'] = label_encoder.fit_transform(df['strength'])

onehot_encoder = OneHotEncoder(sparse_output=False)
strength_encoded = onehot_encoder.fit_transform(df['strength'].values.reshape(-1, 1))

In [5]:
max_length = 20 
X = df['password'].apply(lambda x: [ord(c) for c in x])
X = X.apply(lambda x: np.pad(x, (0, max_length - len(x)), 'constant') if len(x) < max_length else x[:max_length])
X = np.array(X.tolist())
y = strength_encoded

In [6]:
del df

gc.collect()

0

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1441)

In [8]:
del X
del y

gc.collect()

0

In [9]:
model = Sequential()
model.add(Dense(128, input_dim=20, activation='relu'))
model.add(Dense(64, input_dim=20, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))

/media/aghabidareh/b8b65723-1e2e-4a2c-a0f2-1b84824ccd411/projects/a/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1734360103.581095   47599 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4309 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 6GB Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [10]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5


I0000 00:00:1734360106.603751   47762 service.cc:148] XLA service 0x724b3001c590 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734360106.603773   47762 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 6GB Laptop GPU, Compute Capability 8.6
2024-12-16 18:11:46.621622: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1734360106.709907   47762 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-12-16 18:11:46.772794: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version 12.5.82. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


   128/358610 ━━━━━━━━━━━━━━━━━━━━ 7:08 1ms/step - accuracy: 0.5204 - loss: 7.8853     

I0000 00:00:1734360108.013712   47762 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


358610/358610 ━━━━━━━━━━━━━━━━━━━━ 438s 1ms/step - accuracy: 0.9173 - loss: 0.3142 - val_accuracy: 0.9216 - val_loss: 0.3117
Epoch 2/5
358610/358610 ━━━━━━━━━━━━━━━━━━━━ 443s 1ms/step - accuracy: 0.9291 - loss: 0.2718 - val_accuracy: 0.9285 - val_loss: 0.2872
Epoch 3/5
358610/358610 ━━━━━━━━━━━━━━━━━━━━ 435s 1ms/step - accuracy: 0.9291 - loss: 0.2741 - val_accuracy: 0.9302 - val_loss: 0.2823
Epoch 4/5
358610/358610 ━━━━━━━━━━━━━━━━━━━━ 453s 1ms/step - accuracy: 0.9279 - loss: 0.2799 - val_accuracy: 0.9294 - val_loss: 0.2955
Epoch 5/5
358610/358610 ━━━━━━━━━━━━━━━━━━━━ 447s 1ms/step - accuracy: 0.9279 - loss: 0.2839 - val_accuracy: 0.8938 - val_loss: 0.3402


In [12]:
def predict_password_strength(password):
    password_encoded = np.array([ord(c) for c in password]).reshape(1, -1)
    password_encoded = np.pad(password_encoded, ((0, 0), (0, 20 - password_encoded.shape[1])), 'constant')
    prediction = model.predict(password_encoded)
    predicted_class = np.argmax(prediction)
    return label_encoder.inverse_transform([predicted_class])[0]

In [13]:
password = '123456789'
strength = predict_password_strength(password)
print(f"Password Strength: {strength}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
Password Strength: middle


In [14]:
model.save('model.h5')

In [15]:
import pickle
with open('encoder.h5' , 'wb') as f:
    pickle.dump(label_encoder , f)